# Best poet example

Anton Antonov   
August 2025

----

## Setup

In [1]:
use LLM::Graph;
use Graph;
use LLM::Functions;

In [2]:
my $engine = 'neato';

neato

-----

## LLM graph

Nodes spec:

In [3]:
sink my %rules =
    poet1 => "Write a short poem about summer.",
    poet2 => "Write a haiku about winter.",
    poet3 => sub ($topic, $style) {"Write a poem about $topic in the style $style."},
    judge => sub ($poet1, $poet2, $poet3) {
        ["Choose the best composition of your among these:\n\n",
        "1) Poem1: $poet1",
        "2) Poem2: $poet2",
        "3) Poem3: $poet3", 
        "and copy it:"].join("\n\n") }
;

More complicated spec:

In [4]:
sink my %rules =
        poet1 => "Write a short poem about summer.",
        poet2 => "Write a haiku about winter.",
        poet3 => sub ($topic, $style) {
            "Write a poem about $topic in the $style style."
        },
        poet4 => {
                eval-function => {llm-synthesize('You are a famous Russian poet. Write a short poem about playing bears.')},
                test-function => -> $with-russian { $with-russian ~~ Bool:D && $with-russian || $with-russian.Str.lc ∈ <true yes> }
        },
        judge => sub ($poet1, $poet2, $poet3, $poet4) {
            [
                "Choose the composition you think is best among these:\n\n",
                "1) Poem1: $poet1",
                "2) Poem2: $poet2",
                "3) Poem3: {$poet4.defined && $poet4 ?? $poet4 !! $poet3}",
                "and copy it:"
            ].join("\n\n")
        };

Make the graph:

In [5]:
my $gBestPoem = LLM::Graph.new(%rules)

LLM::Graph(size => 5, nodes => judge, poet1, poet2, poet3, poet4)

----

## Full computation

In [ ]:
$gBestPoem.eval(topic => 'Hockey', style => 'limerick', with-russian => 'yes')

In [ ]:
#% html
$gBestPoem.dot(engine => 'dot', :6graph-size, node-width => 1.2):svg

In [ ]:
.say for |$gBestPoem.rules<judge>

In [ ]:
#% html
$gBestPoem.rules.nodemap(*<result>)
==> to-html(align => 'left')